In [ ]:
from pycocotools.cocoeval import COCOeval
import json
import torch
import transform as transf
from torchvision.transforms import Compose
import yaml
from dataset import MedicalBboxDataset
from model import ResNet50
import numpy as np
from utils import bbox_collate, data2target, calc_confusion_matrix, draw_graph

In [4]:
def evaluate_coco_weak(model, val, threshold=0.05):
    config = yaml.safe_load(open('./config.yaml'))
    dataset_means = json.load(open(config['dataset']['mean_file']))
    dataset_all = MedicalBboxDataset(
        config['dataset']['annotation_file'],
        config['dataset']['image_root'])
    if 'class_integration' in config['dataset']:
        dataset_all = dataset_all.integrate_classes(
            config['dataset']['class_integration']['new'],
            config['dataset']['class_integration']['map'])
    
    transform = Compose([
        transf.ToFixedSize([config['inputsize']] * 2),  # inputsize x inputsizeの画像に変換
        transf.Normalize(dataset_means['mean'], dataset_means['std']),
        transf.HWCToCHW()
        ])

    dataset = dataset_all.split(val, config['dataset']['split_file'])
    dataset.set_transform(transform)
    
    model = ResNet50()
    model.load_state_dict(torch.load(f"/data/unagi0/masaoka/wsod/model/resnet50_classify{val}_ulcer.pt"))
    

    results = []
    image_ids = []
    for index in range(len(dataset)):
        data = dataset[index]
        data['img'] = torch.from_numpy(data['img']) #.permute(2, 0, 1)
        # run network
        #scores, labels, boxes = model(data['img'].cuda().float().unsqueeze(dim=0))
        scores, labels, boxes = model(data['img'].unsqueeze(0).float(), e=True, aug=False)
        scores = scores.cpu()
        labels = labels.cpu()
        boxes  = boxes.cpu()

        

        if boxes.shape[0] > 0:
            # change to (x, y, w, h) (MS COCO standard)
            boxes[:, 2] -= boxes[:, 0]
            boxes[:, 3] -= boxes[:, 1]

            # compute predicted labels and scores
            #for box, score, label in zip(boxes[0], scores[0], labels[0]):
            for box_id in range(boxes.shape[0]):
                score = float(scores[box_id])
                label = int(labels[box_id])
                box = boxes[box_id, :]
                

                # scores are sorted, so we can break
                if score < threshold:
                    break

                # append detection for each positively labeled class
                image_result = {
                        'image_id'    : dataset.imgids[index],
                        'category_id' : dataset.label_to_coco_label(label),
                        'score'       : float(score),
                        'bbox'        : box.tolist(),
                    }

                # append detection to results
                results.append(image_result)

        # append image to list of processed images
        image_ids.append(dataset.imgids[index])

        # print progress
        print('{}/{}'.format(index+1, len(dataset)), end='\r')

    if not len(results):
            print("error")
            return
        # write output
    json.dump(results, open(f"/data/unagi0/masaoka/wsod/result_bbox/resnet50_v{val}_ulcer.json", 'w'), indent=4)

    # load results in COCO evaluation tool
    coco_true = dataset.coco
    coco_pred = coco_true.loadRes(f"/data/unagi0/masaoka/wsod/result_bbox/resnet50_v{val}_ulcer.json")

    # run COCO evaluation
    coco_eval = COCOeval(coco_true, coco_pred, 'bbox')
    coco_eval.params.imgIds = image_ids
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    return

In [6]:
evaluate_coco_weak(4)

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!
creating index...
index created!
creating index...
index created!
Loading and preparing results...
DONE (t=0.39s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.53s).
Accumulating evaluation results...
DONE (t=0.87s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.012
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.010
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.082

In [3]:
def evaluate_classify(val, threshold=0.05):
    config = yaml.safe_load(open('./config.yaml'))
    dataset_means = json.load(open(config['dataset']['mean_file']))
    dataset_all = MedicalBboxDataset(
        config['dataset']['annotation_file'],
        config['dataset']['image_root'])
    if 'class_integration' in config['dataset']:
        dataset_all = dataset_all.integrate_classes(
            config['dataset']['class_integration']['new'],
            config['dataset']['class_integration']['map'])
    
    transform = Compose([
        transf.ToFixedSize([config['inputsize']] * 2),  # inputsize x inputsizeの画像に変換
        transf.Normalize(dataset_means['mean'], dataset_means['std']),
        transf.HWCToCHW()
        ])

    dataset = dataset_all.split(val, config['dataset']['split_file'])
    dataset.set_transform(transform)
    dataloader_val = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=False, 
                                                num_workers=4, collate_fn=bbox_collate)
    model = ResNet50()
    model.load_state_dict(torch.load(f"/data/unagi0/masaoka/wsod/model/resnet50_classify1.pt"))
    ite = 0
    gt = [0,0,0,0]
    tpa = np.zeros(3)
    fpa = np.zeros(3)
    tna = 0
    fna = np.zeros(3)
    with torch.no_grad():
        for i, d in enumerate(dataloader_val):
            scores = torch.sigmoid(model(d['img'].cuda().float()))
            output = scores.cpu().data.numpy()
            output = np.where(output>0.5,1,0)
            target, n, t, v, u = data2target(d, torch.from_numpy(output))
            target = target.cpu().data.numpy()
            gt = np.array([n,t,v,u])
            tp, fp, fn, tn = calc_confusion_matrix(output, target, gt)
            tpa += tp
            fpa += fp
            fna += fn
            tna += tn
                
            print(f'{i}/{len(dataloader_val)}', end = '\r')
    print(gt,tpa,fpa,tna,fna)
    return gt,tpa,fpa,tna,fna